# 01_Geoviews_WorldMETAR.ipynb

## Overview:
1. Create an interactive visualization of worldwide METAR data using the [Holoviz](https://holoviz.org) ecosystem

## Imports

In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
import geoviews as gv
import geoviews.feature as gf
from geoviews import opts
import geoviews.tile_sources as gts
from cartopy import crs as ccrs

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| Pandas  | Necessary | |
| Contextily  | Helpful| |

* **Time to learn**: 20 minutes
***

## Create an interactive visualization of worldwide surface meteorological (METAR) data using the Holoviz ecosystem

[Holoviz](https://holoviz.org) is a suite of open-source Python libraries designed for interactive data analysis and visualization via the browser (including the Jupyter notebook). In this notebook, we will use the [GeoViews](http://geoviews.org/) package, which is part of Holoviz.

Another part of the Holoviz ecosystem is [bokeh](https://bokeh.org/). Bokeh leverages Javascript in order to accomplish interactivity via the browser. GeoViews makes available Bokeh as well as Matplotlib via an extenstion. 

In [ ]:
gv.extension('bokeh', 'matplotlib')

## Use Pandas to read in the file containing the most recent METAR data. 
Since this file has latitude and longitude, we can pass the dataframe directly to GeoViews (i.e. no need to use Geopandas).

In [ ]:
# First define the format and then define the function
timeFormat = "%y%m%d/%H%M"
# This function will iterate over each string in a 1-d array
# and use Pandas' implementation of strptime to convert the string into a datetime object.
parseTime = lambda x: datetime.strptime(x, timeFormat)

df = pd.read_csv('/spare11/atm533/data/world_metar_latest.csv',parse_dates=['YYMMDD/HHMM'], date_parser=parseTime, sep='\s+')

In [ ]:
df.columns

In [ ]:
df.head(5)

In this dataset, missing values are set to -9999.0. Let's replace any instance of this value with `np.nan` throughout the DataFrame.

In [ ]:
df.replace(-9999.0,np.nan,inplace=True)

In [ ]:
df.head(5)

We'll use pandas' `describe` function to find the max, min, mean, and other basic statistical properties of one of the `Series` in the `DataFrame`.

In [ ]:
df.TMPC.describe()

## Create a set of GeoViews `Points` objects. 

We pass it three arguments: 
1. The Pandas dataframe
2. A list containing the lons and lats
3. A list containing the columns we want to include

In [ ]:
df_points = gv.Points(df, ['SLON','SLAT'],['STN','TMPC','PMSL','DWPC','SPED','P01M'])

Visualize the GeoViews Points object

In [ ]:
df_points

Notice that the x and y-axes correspond to the range of lons and lats in the dataframe. Also notice the Toolbar to the right of the plot. You can mouse over each tool to see its function. By default, it is in Pan mode ... click and drag to move around.

Next, activate the *Box Zoom* tool (the single magnifying glass). **Click and drag** from **upper-left to lower-right** to create a box. The plot will automatically adjust to the new zoomed-in view.

If you have a mouse, you can also try the *Wheel Zoom* tool, which is just below the *Box Zoom* tool.

The *Reset* tool (i.e., the bottom-most icon) resets the plot to the full domain.

## Geo-reference the image via a background raster image.
The Holoviz suite uses the * to add layers to the same plot. Here, we first specify the Open Street Maps tile source. Then, we add our Points dataframe to it. We also specify options that accomplish the following:
1. Specify the width and height of the plot
1. Set the size of the points
1. Color each point by a variable: in this case, 2 m temperature in Celsius
1. Add a colorbar
1. Add the `hover` tool to the Toolbar

In [ ]:
(gv.tile_sources.OSM * df_points).opts(
    opts.Points(frame_width=800, frame_height=600, size=8, color='TMPC',tools=['hover']))

Note that `hover` tool is active and appears as the tool icon just below the `reset` tool. Mouse over any of the points and you will see a readout of the data from all the columns that we included in the creation of the df_points GeoViews object.

<div class="alert alert-warning">
    <b>Note: </b> Some of the variables, such as latitude (<b>SLAT</b>) and longitude (<b>SLON</b>), may have non-sensical values ... another reminder that the <b>Holoviz</b> suite is still a work in progress!
        </div>

## Make a `Labels` plot

Next, let's plot a map that instead of the point locations, outputs the values of a particular column from the dataset. To do this, we create a GeoViews `Labels` object. It takes arguments similar to `Points`.

In [ ]:
df_labels = gv.Labels(df, ['SLON','SLAT'],['TMPC'])

Plot just the labels.

In [ ]:
df_labels

Now layer on the background map tile, as we did for `Points`. Specify the color and size of the text labels.

In [ ]:
figure = (gv.tile_sources.CartoLight * df_labels).opts(
    opts.Labels(frame_height=800,frame_width=800,text_color='purple',text_font_size='10pt'))

In [ ]:
figure

<div class="alert alert-warning">
    <b>Note: </b> If you use the panning tool such that you go beyond the bounds of the map, the data does not reappear.
        </div>

## What basemaps do we have available "built-in" in Geoviews?

In [ ]:
gv.Layout([ts.relabel(name) for name, ts in
    gts.tile_sources.items()]).opts(
        'WMTS', xaxis=None, yaxis=None, width=225, height=225).cols(6)

<div class="alert alert-warning">NOTE: if you are running this via Jupyterhub, some of the maps above (e.g. Wikipedia) will be *blank*. This is related to the fact that while the notebook is being displayed on your local computing device, the request to the tile server is coming from the Jupyterhub server, whose IP address is different from your local device. Some tile servers are configured in a way that forbids this type of web request, in order to guard against <a href="https://en.wikipedia.org/wiki/Cross-site_request_forgery">cross-site request forgery</a>. Future releases of Jupyterhub should have some methods to deal with this issue.

<div class="alert alert-warning">
    <b>Things to try:</b> <br>
<ol><li>Plot another data source instead of (or in addition to) the worldwide METAR observations.</li>
    <li>Plot a variable other than temperature.</li></ol></div>

---
## Summary
* The *Holoviz* set of Python packages provide interactive visualization of datasets in the Jupyter notebook.
* Data from Pandas dataframes can be easily displayed and geo-referenced by *Geoviews* `Point` and `Label` objects.
* Similar to *Contextily*, *Geoviews* allows one to add background tile-served maps as an additional layer.


## References:
1. [Holoviz](https://holoviz.org)
1. [Geoviews](https://geoviews.org)